# Data Analysis Notebook for Neuronal Differentation 2021

Authored by Abhranil Maiti (Riju) - MSc Bioinformatics and Theoretical Systems Biology, Imperial College London

After the gene counts have been created from the raw reads (see Bash notebook for Neuronal Differentation 2021 for more information), the next step is the data analysis and interpretation! 

For this notebook, you will need the following dependencies:
- R-Studio/R/R-kernel (if using this Jupyter Notebook)
- DESeq2
- ggplot2

The files required for this notebook are:
- Read Count files (**NOT** TPMs)
- Metadata.txt file

More information is found in Part 2

This section will be split into four parts:

- Part 1: Package installation (optional)
- Part 2: Data Preparation
- Part 3: DESeq2 Processing
- Part 4: Data Analysis

If any part of the Jupyter Notebook appears to be out of date or is incorrect, or if there are any questions, please contact the author at: abhranil.maiti20@imperial.ac.uk, or on GitHub (riju.maiti).

# Part 1: Package installation (optional)

This step is optional if you do not have the required packages specified above. They can be installed by running the code-block/copying the code below.

DESeq2 is part of the Bioconductor suite of programs. Thus, installation is quite straightforward as shown below:

In [ ]:
#checks if R/R-Studio has BiocManager already installed. If not, it will install BiocManager. 
#Biocmanager is used for many other bioinformatics packages so it is useful to install

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.12")

#note: version 3.12 is the latest as of 23/04/2021

#using BiocManager, installs DeSeq2 on the machine. 

BiocManager::install("DESeq2")

ggplot2 and ggpubr are also quite useful for plot visualisation, so it is also recommended to install these (will be used)

In [ ]:
install.packages("ggplot2")
install.packages("ggpubr")

Don't forget to also load the libraries once installed!

In [ ]:
library("DESeq2")
library("ggplot2")
library("ggpubr")

# Part 2: Data Preparation
Like with all data analysis software, there is an element of data preparation before it runs wild and generates some inferences. Here are the first few steps:

### **2.1** Preparing the raw counts file

Lets first load the raw gene counts file, as shown here:

In [ ]:
#list your directory where your raw gene count files are located in the square brackets
#remember to remove the square brackets after!
loc_df <- "[your-directory-here]"

#reads in the csv file into the dataframe
#if using a Tab Delimited File, use "\t" instead of "," in the "sep" field
df <- read.table(loc_df, row.names=,1, sep=",")

The Raw Gene Counts files should be obtained from the "Bash notebook for Neuronal Differentation 2021" notebook

If we look inside this file however, we can see a problem:

In [ ]:
head(df)

The gene count files that have been created will contain the Ensembl IDs as well as the raw gene counts. Unfortunately, the Ensembl IDs will have their version numbers, which may cause confusion to the DESeq program downstream! To remedy this, we can use a simple R-script line as shown below:

In [ ]:
#the term after the $ refers to the column that your ensembl_IDs are located in
#by default this should be titled "gene_id"
df$gene_id <- gsub("\\..*","",df$gene_id)

We can then save the files using the following command below. This was done for both mouse and human samples.

In [ ]:
write.csv(merged, "[your_counts_file].csv")

### **2.2** Preparing the metadata.txt file

With every experiment, there are usually several replicates. However, we need to specify to DESeq which samples are biological replicates, and which samples are replicates for different conditions! 

The format for the metadata.txt file is quite easy. Using our data for instance, they were laid out as follows (mouse):

Day 0: m_0_1, m_0_2, m_0_3

Day 0.5: m_0.5_1, m_0.5_2, m_0.5_3

Day 1: m_1_1, m_1_2, m_1_3
etc. 

Therefore, the metadata.txt file would look as follows:

<br>sample &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; group
<br>m_0_1  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  day_0
<br>m_0_2  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  day_0
<br>m_0_3  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  day_0
<br>m_0.5_1 &nbsp; &nbsp; &nbsp; day_0.5
<br>m_0.5_2 &nbsp; &nbsp; &nbsp; day_0.5
<br>m_0.5_3 &nbsp; &nbsp; &nbsp; day_0.5
<br>m_1_1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;   day_1
<br>m_1_2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;   day_1
<br>m_1_3 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;   day_1
<br>etc. 

The metadata.txt files can be created using Microsoft Excel, and saved as a "Text (Tab Delimited) (.txt)". This option is proposed in Microsoft Excel

**IMPORTANT** You cannot have multiple spaces between words when creating your groups/samples. In the example above, "Day 0" would need to be converted to "Day_0".

**IMPORTANT** Your metadata.txt file must be ordered in the same way that your headings are saved as. For instance, an excerpt of our human data headings were as follows:

... h_11_2   h_11_3   h_1_1   h_12_1  h_12_2  h_12_3  h_1_2 ...

Therefore, the metadata needed to be structured as follows:

<br>sample &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; group
<br>h_11_2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 11
<br>h_11_3 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 11
<br>h_1_1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 1
<br>h_12_1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 12
<br>h_12_2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 12
<br>h_12_3 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 12
<br>h_1_2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 1

After doing that, you can simply load the metadata.txt file into R

In [ ]:
#list your directory where your metadata.txt file is located in the square brackets
#remember to remove the square brackets after!
loc_metadata <- "[your-directory-here]"

#reads in the metadata file into the dataframe
metadata <- read.table(loc_metadata, row.names=,1) 

For our DESeq2 analysis, we combined the mouse and human datasets together. Therefore, in the metadata, you need another column titled "species" so that the program can distinguish the samples that came from mouse, and the samples that came from human. See below for example:

<br>sample &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; group &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; species
<br>m_0_1  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  day_0 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; mouse
<br>m_0_2  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  day_0 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; mouse
<br>m_0_3  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  day_0 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; mouse
<br>h_12_1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 12 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; human
<br>h_12_2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 12 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; human
<br>h_12_3 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; day 12 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; human

Don't forget, the metadata must be ordered in the same way the headings that are found in the file!

### **2.3** Orthologous Gene Extraction

Next, we extracted orthologous genes so that sample comparisons between mouse and humans. This is optional for other analysis, but important for species comparisons. The code below was used. 

NOTE: List of Orthologs was obtained from BioMart (https://www.ensembl.org/biomart/martview/) and the following fields selected: "Gene Stable ID", "Gene Name"

In [ ]:
#inputting the files saved earlier for both mouse and human counts
df_human <- read.csv("[file_containing_human_counts.tsv", sep="\t", header=TRUE)
df_mouse <- read.csv("[filee_containing_mouse_counts.tsv", sep = "\t", header=TRUE)

#inputting the files detailing the mouse orthologs to human genes, and vice versa
#two DISTINCT files are needed as one file will have human ENSEMBL gene IDs
#the other file will have mouse ENSEMBL IDs
df_orthologs_mouse <- read.csv("human_mouse_orthologs.txt", sep="\t", header=TRUE)
df_orthologs_human <- read.csv("mouse_human_orthologs.txt", sep="\t", header=TRUE)

#the ortholog gene list is merged ONLY if there are matching gene IDs in both files
#gene IDs that are not found in either list are omitted.
#if the final dataframe output has nothing in it, check that the orthologs list you are merging
#hasn't been switched

merged_human <- merge(df_human, data_orthologs_human, by.x="Gene.stable.ID", by.y="Gene", all=FALSE)
merged_mouse <- merge(df_mouse, data_orthologs_mouse, by.x="Gene.stable.ID", by.y="Gene", all=FALSE)

#next, both mouse and human files were to make one big file containing both samples:
#for the merge to be successful, the mouse gene names need to be capitalised
merged_master <- merge(df_human, data_mouse, by.x="Gene.Name", by.y="Gene.Name", all=FALSE)

#files can then be written to a csv file after
write.csv(merged_master, "DESEq2_ready_file.csv")


# Part 3: DESeq2 processing

Once the files have been setup and loaded, DESeq2 can be run.

Note: Prior to this step, the ENSEMBL ID columns were deleted. The remaining columns should just be one column with gene names, and the samples containing raw gene counts.

Before doing so, we can perform an (optional) step of excluding genes whos counts are less than 10 overall, or genes that have no recorded expression. This can be done by executing the code below:

In [ ]:
#removes genes who do not have any recorded expression at all.
#you can change 0 to be anything i.e. 10 to only include genes that have a summed count of 10 or more for a particular gene.
merged_master <- merged_master[rowSums(merged_master[, -1])>0, ]

This step isn't always necessary, but it does speed up the processing time for DESeq2 in case you do have a large number of samples.

Next, a DESeq2 matrix is created. This is done as follows:

In [ ]:
#creates a summarised DESeq2 matrix
dds <- DESeqDataSetFromMatrix(countData=merged_master, colData=metadata, design=~group+species)

We then need to run DESeq2:

In [ ]:
#command to run the differential gene expression analysis
dds <- DESeq(dds)

As we are concerned with doing network analysis, we will only be using DESeq2 to generate homoskedastic data i.e. variance does not depend on the mean. We need to do this as in RNASeq data, the mean grows with the variance. The code is as below:

In [ ]:
newdata <- getVarianceStabilizedData(dds) 

We can then save our normalised data as shown here:

In [ ]:
#writes a csv file of your variance stabilized data. Change the file name as is required. 
write.csv(newdata, "[your file name here]")

# Part 4: Data Analysis

Now for the fun part; data analysis! This is where you can finally view just how pretty the data is.

This section is split into three parts (based on the project):

*   Generating Plots
*   Principal Component Analysis
*   Time-Factor Estimations





### **3.1** Generating Plots

For this section, we need the following packages: ggplot2, and ggpubr.

ggplot2 does the heavy lifting in terms of plot generation. 
ggpubr arranges the plots nicely for publications.

Prior to running the code, Pax6, Irx3, Nkx2.2, and Olig2.2 were extracted using Excel, data feature-scaled (normalised between 0 and 1) and then values were averaged. The file format was as follows:

<br>day &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Pax6 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Olig2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Nkx2.2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Irx3
<br>0.5  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.00286 &nbsp; &nbsp; &nbsp;
0.00024 &nbsp; &nbsp;
0.00010 &nbsp; &nbsp; &nbsp; &nbsp;
0.0723
<br>1  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  0.84609  &nbsp; &nbsp; &nbsp;
0.22127 &nbsp; &nbsp;
0.01384 &nbsp; &nbsp; &nbsp; &nbsp;
0.89926
<br>2  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 0.49591 &nbsp; &nbsp; &nbsp;
0.54934 &nbsp; &nbsp; 
0.04497 &nbsp; &nbsp; &nbsp; &nbsp;
0.82328
<br> etc.


For TPM vs VST data, the following code was used:

In [ ]:
#example code for reading in a file containing processed data as described above.
df_mouse_TPM <- read.csv("[data_for_mouse_in_TPM.csv]", sep=",", header=TRUE)
df_mouse_VST <- read.csv("[data_for_mouse_in_VST.csv]", sep=",", header=TRUE)

df_human_TPM <- read.csv("[data_for_human_in_TPM.csv]", sep=",", header=TRUE)
df_human_VST <- read.csv("[data_for_human_in_VST.csv]", sep=",", header=TRUE)

#breaking down an example
TPM_mouse <- ggplot(df_mouse_TPM, aes(x = Day)) + #defines a new ggplot using mouse_TPMs
  geom_line(aes(y = Pax6, color = "Pax6"), size = 1.2) + #defines the Pax6 line using Pax6 data from the dataframe
  geom_line(aes(y = Nkx2.2, color = "Nkx2.2"), size = 1.2) + #defines the Nkx2.2 line using Pax6 data from the dataframe
  geom_line(aes(y = Olig2 color = "Olig2"), size = 1.2) + #defines the Olig2 line using Pax6 data from the dataframe
  geom_line(aes(y = Irx3, color = "Irx3"), size = 1.2) + #defines the Irx3 line using Pax6 data from the dataframe
  ggtitle("Mouse GRN (TPM)") + #creates a title for the plot
  labs(x = "Days in RA + SAG", y = "Normalised Expression", size = 15) + #labels the axes
  xlim(0, 7) + #sets the limit of the x-axis
  
  theme(axis.line = element_line("black"), panel.background = element_rect(fill = 'white'),  #changes the background to be white and the axis lines grey
        legend.title=element_blank(), legend.text = element_text(size = 15), #removes the legend title, and makes the legend text size 15
        plot.title = element_text(size=15, face="bold.italic", hjust = 0.5), #makes the plot title 15, bold.italic, and centres the title
        text = element_text(size = 15)) + #makes the axis text size 15
  
  scale_colour_manual(values=c("#2ca02c", "#1f77b4", "#d62728", "#ff7f0e")) #sets the colour of the Pax6, Irx3, Olig2, and Nk2.2 lines

TPM_human <- ggplot(df_human_TPM, aes(x = Day)) +
  geom_line(aes(y = Pax6, color = "Pax6"), size = 1.2) +
  geom_line(aes(y = Nkx2.2, color = "Nkx2.2"), size = 1.2) +
  geom_line(aes(y = Olig2, color = "Olig2"), size = 1.2) +
  geom_line(aes(y = Irx3, color = "Irx3"), size = 1.2) +
  ggtitle("Human GRN (TPM)") +
  labs(x = "Days in RA + SAG", y = "Normalised Expression", size = 15) +
  xlim(0, 15) +
  
  theme(axis.line = element_line("black"), panel.background = element_rect(fill = 'white'), 
        legend.title=element_blank(), legend.text = element_text(size = 15),
        plot.title = element_text(size=15, face="bold.italic", hjust = 0.5), 
        text = element_text(size = 15)) +
  
  scale_colour_manual(values=c("#2ca02c", "#1f77b4", "#d62728", "#ff7f0e"))

VST_mouse <- ggplot(df_mouse_VST, aes(x = Day)) +
  geom_line(aes(y = Pax6, color = "Pax6"), size = 1.2) +
  geom_line(aes(y = Nkx2.2, color = "Nkx2.2"), size = 1.2) +
  geom_line(aes(y = Olig2, color = "Olig2"), size = 1.2) +
  geom_line(aes(y = Irx3, color = "Irx3"), size = 1.2) +
  ggtitle("Mouse GRN (VST)") +
  labs(x = "Days in RA + SAG", y = "Normalised Expression", size = 15) +
  xlim(0, 7) +
  
  theme(axis.line = element_line("black"), panel.background = element_rect(fill = 'white'), 
        legend.title=element_blank(), legend.text = element_text(size = 15),
        plot.title = element_text(size=15, face="bold.italic", hjust = 0.5), 
        text = element_text(size = 15)) +
  
  scale_colour_manual(values=c("#2ca02c", "#1f77b4", "#d62728", "#ff7f0e"))

VST_human <- ggplot(df_human_VST, aes(x = Day)) +
  geom_line(aes(y = Pax6, color = "Pax6"), size = 1.2) +
  geom_line(aes(y = Nkx2.2, color = "Nkx2.2"), size = 1.2) +
  geom_line(aes(y = Olig2, color = "Olig2"), size = 1.2) +
  geom_line(aes(y = Irx3, color = "Irx3"), size = 1.2) +
  ggtitle("Human GRN (VST)") +
  labs(x = "Days in RA + SAG", y = "Normalised Expression", size = 15) +
  xlim(0, 15) +
  
  theme(axis.line = element_line("black"), panel.background = element_rect(fill = 'white'), 
        legend.title=element_blank(), legend.text = element_text(size = 15),
        plot.title = element_text(size=15, face="bold.italic", hjust = 0.5), 
        text = element_text(size = 15)) +
  
  scale_colour_manual(values=c("#2ca02c", "#1f77b4", "#d62728", "#ff7f0e"))

#this function from ggpubr arranges the plots together, and combines the legend and displays it at the bottom of the plots
  ggarrange(TPM_human, VST_human, TPM_mouse, VST_mouse, labels = c("A", "B", "C", "D"), common.legend = TRUE, legend = "bottom")


### **3.2** Principal Component Analysis
For this section, we need the following packages: ggplot2, DESeq2

We can just use the Variance Stabilised Data that had been created previously, as well as the following code.

In [ ]:
pcaData <- plotPCA(newdata, intgroup=c("group", "species"), returnData=TRUE) #group is specified depending on the headings in the metadata.txt
#this function calculates the variance of each gene in the dataset before doing the dimensionality reduction

ggplot(pcaData, aes(PC1, PC2, color=group.1, shape=species)) + #creates the plot using PC1 and PC2 found in "pcaData"
  geom_point(size=3) + #sets the size  of the data points
  xlab(paste0("PC1: ",percentVar[1],"% variance")) + #sets the x-label
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + #sets the y-label
  theme(axis.line = element_line("black"), panel.background = element_rect(fill = 'white')) + #sets the background
  coord_fixed()

  #additionally, the plots should automatically generate the legends, and represent the different species and time-points using different shapes/colours
  #this is to distinguish the samples from one another.

#writes a csv file of PCA data for both mouse and human. Change the file name as is required. 
write.csv(pcaData, "[your file name here]")

### **3.3** Time Factor Estimations
For this section, we need the following packages: ggplot2, DESeq2

Firstly, a file comparing PC2 and Days (of experiment) was prepared in Excel for mouse and human data. Format was below:

<br>Day &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; PC2 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; species
<br>0 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; -40.20901 &nbsp; &nbsp;  human 
<br>0 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; -40.54826 &nbsp; &nbsp;  human 
<br>0 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; -39.96980 &nbsp; &nbsp;  human 
<br>1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; -32.17217 &nbsp; &nbsp;  human 
<br>1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; -32.31412 &nbsp; &nbsp;  human 
<br>1 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; -32.01572 &nbsp; &nbsp;  human 
<br> etc.

Then, the regression analysis was carried out as follows:

In [ ]:
#reads in the dataframes for both mouse and human
df <- read.csv("[data_for_mouse_human_PC2.csv]", sep=",", header=TRUE)

df_mouse <- df[49:81, ] #extracting only mouse data
df_human <- df[4:45, ] #extracting only human data

#IMPORTANT: 0 was excluded from regression analysis

#assigns variables for regression analysis
x.mouse <- df_mouse$Day
y.mouse <- df_mouse$PC2.

x.human <- df_human$Day
y.human <- df_human$PC2.

#runs the regression analysis
model.mouse <- lm(y.mouse ~ log(x.mouse))
summary(model.mouse)

model.human <- lm(y.human ~ log(x.human))
summary(model.human)

From the regression analysis, the data points could then be plotted, and the regression curves obtained from the regression analysis can also be overlayed.

In [ ]:
fun.1 <- function(x) 28.198*log(x) - 20.246 #equation based off the regression analysis run for mouse data
fun.2 <- function(x) 19.8707*log(x) - 37.626 #equation based off the regression analysis run for human data

#functions were then plotted
ggplot(df, aes(x = Day, y = PC2..Time., group = species, color = species)) + #creates initial plots
  geom_point(stat="identity") + #adds each datapoint to the graph
  labs(x = "Days in RA + SAG", y = "Gene Expression Profile (PC2)") + #sets the labels of the axes
  theme(axis.line = element_line("black"), panel.background = element_rect(fill = 'white'), #makes the background white and axes black
        legend.title=element_blank(), legend.text = element_text(size = 16), axis.title=element_text(size=16), #sets the size of the text
     axis.text=element_text(size=16)) +

  geom_function(fun = fun.2, color="#F8766D", size = 1.2) + #sets the colour of the mouse function
  geom_function(fun = fun.1, color="#00BFC4", size = 1.2) + #sets the colour of the human function
  xlim(0.5, 15) #sets the limit of the graphs